In [1]:
import importlib
import pandas as pd
import numpy as np
import re

from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

pd.set_option("display.max_colwidth", 200)

In [7]:
"""
Завантажити базу закупівель ProZorro (деталі у README.txt) і створити список з назвами колонок
"""
df_prozorro = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)
training_data_columns = df_prozorro.columns.tolist()

In [8]:
"""
Завантажити датафрейм МОН навчальниз закладів, підготовлений у "Fileter MON data.ipynb"
"""
df_mon_data_bez_upravlin_osvity = pandas.read_csv('data/df_bez_upravlin_osvity.csv', sep = ';')

In [9]:
"""
Створити список унікальних ЄДРПОУ освітніх закладів МОН
"""
mon_data_list = df_mon_data_bez_upravlin_osvity.edrpou.unique().tolist()

In [ ]:
# # CODE FOR DEBUGGING PURPOSES

# COLUMNS_TO_SHOW = ['']
# VALUE_FILTER = ''
# COLUMN_TO_FILTER = ''
# debug_df = PUT_DARAFRAME_NAME_HERE[COLUMNS_TO_SHOW]
# debug_df[debug_df[COLUMN_TO_FILTER] == VALUE_FILTER]

### ШКОЛИ ЗАМОВНИКИ

##### Step 1: Створити базу навчальних закладів, які самостійно купують через ProZorro

In [10]:
"""
Відфільтрувати базу закупівель df_prozorro за колонкою IDOrganizator.
Фільтром є списко МОН з ЄДРПОУ начальних закладів mon_data_list (попередній крок)

df_tenders_by_schools - база закупівель, де навчальні заклади є замовниками
df_undefined_1 - база невизначених закупівель ProZorro
"""
df_tenders_by_schools = df_prozorro[df_prozorro['IDOrganizator'].isin(mon_data_list)]
df_undefined_1 = df_prozorro[~df_prozorro['IDOrganizator'].isin(mon_data_list)]

##### Step 2: Класифікація шкіл-замовників по типах за колонкою "Организатор"

In [11]:
df_tenders_by_schools_classified = classification_by_type_one_column(df_tenders_by_schools,
                                                                     ['Организатор'], 
                                                                     filter_priority, 
                                                                     filter_name_priority)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Тип закладу'] = "Невідомо"
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df[column + "_check"] = ""
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:60: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  case = False, na = False, regex = True)
/home/a

##### Step 3: Нумерація шкіл-замовників

In [12]:
df_tenders_by_schools_numerated = numeration_one_column(df_tenders_by_schools_classified, 
                                                        ['Организатор'], 
                                                        reg_number)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Номер'] = given_df[column].str.findall(f"{regex}", flags = re.IGNORECASE).apply(''.join)


##### Step 4: Найменування шкіл-замовників

In [13]:
df_tenders_by_schools_named = naming_one_column(df_tenders_by_schools_numerated, 
                                                ['Организатор'], 
                                                reg_quotes)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Назва'] = given_df[column].str.findall(f"{regex}", flags = re.IGNORECASE).str[0].str[-1]


##### Step 5: Додаткове фільтрування бази шкіл-замовників і створення фінальної бази ШКІЛ ЗАМОВНИКІВ

In [14]:
"""
Видалення закупівель, де зустрічаються ключові слова filter_poza
"""

df_tenders_by_schools_named = df_tenders_by_schools_named[~df_tenders_by_schools_named['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_poza)),
                case = False, na = False, regex = True)]
# перетворити пусті строки в NaN
df_tenders_by_schools_named['Номер'] = df_tenders_by_schools_named['Номер'].replace(r'^\s*$', np.nan, regex=True)

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
"""
Видалення "технічних" колонок, створених функціями.
Створення колонки 'Заклад-замовник'
"""
df_tenders_by_schools = df_tenders_by_schools_named.copy()
df_tenders_by_schools['Заклад-замовник'] = True
col_names = training_data_columns.copy() 
col_names += ['Заклад-замовник', 'Тип закладу', 'Номер', 'Назва']

In [16]:
"""
df_tenders_by_schools - база закупівель, де замовниками є навчальні заклади

"Назва закладу" - колонка з фіналізованою назвою школи. Принцип визначення:
    1. Якщо колонка "Номер" не є пустою - додається до "Типу закладу"
    2. Якщо колонка "Номер" є пустою - "Назва закладу" отримує значення "Организатор"

* колонка "Назва" не враховується через високий ступінь повторюваності з "Организатор" і часткову точність
"""
df_tenders_by_schools = df_tenders_by_schools[col_names]
df_tenders_by_schools['Назва закладу'] = np.where(df_tenders_by_schools['Номер'].notnull(), df_tenders_by_schools['Тип закладу'] + ' ' + df_tenders_by_schools['Номер'], 
                                                 df_tenders_by_schools['Организатор'])

### ШКОЛИ НЕЗАМОВНИКИ

##### Step 5: Фільтрування закупівель шкіл-не-замовників за допомогою пошуку ключових слів в колонках

In [17]:
"""
З-поміж датафрейму невизначених закупівель df_undefined_1 за ключовими словами у списках
filter_keywords, filter_keywords_strict відбираються закупівлі освітніх закладів.

Ключові слова шукаються у трьох колонках: Тендер, ОписаниеТендера, Лот 

Примітка: регулярний вираз /b{}/b шукає за ключовими словами зі списку filter_keywords_strict_extended тільки 
ті паттерни, де ці ключові слова є окремими, а не зустрічають всередині слів. Це зроблено з метою зменшення
кількості "брудних" даних
"""
df_tenders_not_by_schools_by_keywords = df_undefined_1[df_undefined_1['Тендер'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True)  | 
                                                      df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | 
                                                      df_undefined_1['Лот'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | df_undefined_1['Тендер'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | df_undefined_1['Лот'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True)]

/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/artem/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


##### Step 6: Класифікація шкіл-не-замовників за типами

In [18]:
df_tenders_not_by_schools_classified = classification_by_type_three_cols(df_tenders_not_by_schools_by_keywords,
                                                                         ['Тендер', 'ОписаниеТендера', 'Лот'], 
                                                                         filter_priority, 
                                                                         filter_name_priority)

/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df['Тип закладу'] = "Невідомо"
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  given_df[column + "_check"] = ""
/home/artem/projects/tmp/school-procurements/functions/functions_dataframe.py:40: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  case = False, na = False, regex = True)
/home/a

##### Step 7: Нумерація шкіл-не-замовників

In [ ]:
"""
Із закупівель df_tenders_not_by_schools_classified, де був успішно визначений "Тип закладу" (немає значення "Невідомо"),
створюється датафрейм 
"""
df_step7 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()

In [4]:
df_tenders_not_by_schools_numerated, df_step7_numerated_undefined_4 = long_numeration(df_step7, reg_number)

## END EXPERIMETNS

In [ ]:
"""
Із закупівель df_tenders_not_by_schools_classified, де був успішно визначений "Тип закладу" (немає значення "Невідомо"),
створюється датафрейм df_step7
"""
df_step7 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()

In [ ]:
"""
Принцип фільтрування в функції numeration():
    Визначається наявністья символу № в колонках 'Тендер', 'Лот', 'ОписаниеТендера'.
    Регулярний вираз знаходить всі паттерни і джойнить через кому.
    Паралельно створюється колонка, де рахується кількість знайдених паттернів.
    
df_step7_numerated - результат фільтрування numeration()
"""
df_step7['ОписаниеТендера'] = df_step7['ОписаниеТендера'].fillna('') # Технічний крок для коректної роботи функції numeration 
df_step7_numerated = numeration(df_step7, ['Тендер', 'Лот', 'ОписаниеТендера'], reg_number, 'numeration')

In [ ]:
"""
df_step7_numerated_lot - датафрейм, де в колонці Лот знайдений один паттерн
Оновлення колонки "Номер".
df_step7_numerated_undefined_2 - решта неідентифікованих закупівель 
"""
df_step7_numerated_lot_tender = df_step7_numerated[df_step7_numerated['Номер'] != 'Невідомо']
df_step7_numerated_undefined = df_step7_numerated[df_step7_numerated['Номер'] == 'Невідомо']

In [ ]:
"""
df_step7_numerated_lot - датафрейм, де в колонці Лот знайдений один паттерн
Оновлення колонки "Номер".
df_step7_numerated_undefined_2 - решта неідентифікованих закупівель 
"""
df_step7_numerated_lot = df_step7_numerated_undefined.copy()
df_step7_numerated_lot['Номер'] = np.where(df_step7_numerated_lot['Лот_numeration_count'] == 1, 
                                                 df_step7_numerated_lot['Лот_numeration'], 
                                                 'Невідомо')
df_step7_numerated_undefined_2 = df_step7_numerated_lot[df_step7_numerated_lot['Номер'] == 'Невідомо']
df_step7_numerated_lot = df_step7_numerated_lot[df_step7_numerated_lot['Номер'] != 'Невідомо']

In [ ]:
"""
df_step7_numerated_opystendera - датафрейм, де в колонці Описаниетендера знайдений один паттерн
Оновлення колонки "Номер".
df_step7_numerated_undefined_3 - решта неідентифікованих закупівель 
"""
df_step7_numerated_opystendera = df_step7_numerated_undefined_2.copy()
df_step7_numerated_opystendera['Номер'] = np.where(df_step7_numerated_opystendera['ОписаниеТендера_numeration_count'] == 1, 
                                                 df_step7_numerated_opystendera['ОписаниеТендера_numeration'], 
                                                 'Невідомо')
df_step7_numerated_undefined_3 = df_step7_numerated_opystendera[df_step7_numerated_opystendera['Номер'] == 'Невідомо']
df_step7_numerated_opystendera = df_step7_numerated_opystendera[df_step7_numerated_opystendera['Номер'] != 'Невідомо']

In [ ]:
"""
df_step7_numerated_opystendera - датафрейм, де в колонці  Тендер знайдений один паттерн
Оновлення колонки "Номер".    
df_step7_numerated_undefined_4 - решта неідентифікованих закупівель 
"""
df_step7_numerated_tender = df_step7_numerated_undefined_3.copy()
df_step7_numerated_tender['Номер'] = np.where(df_step7_numerated_tender['Тендер_numeration_count'] == 1, 
                                                 df_step7_numerated_tender['Тендер_numeration'], 
                                                 'Невідомо')
df_step7_numerated_undefined_4 = df_step7_numerated_tender[df_step7_numerated_tender['Номер'] == 'Невідомо']
df_step7_numerated_tender = df_step7_numerated_tender[df_step7_numerated_tender['Номер'] != 'Невідомо']

In [ ]:
"""
df_tenders_not_by_schools_numerated - об'єднаний датафрейм шкіл-не-замовників з номерами.
Видалення "технічних" колонок, створених функціями.
"""
frames = [
            df_step7_numerated_lot_tender, 
            df_step7_numerated_lot, 
            df_step7_numerated_opystendera, 
            df_step7_numerated_tender
         ]
df_tenders_not_by_schools_numerated = pd.concat(frames)
# df_tenders_not_by_schools_numerated['Назва'] = np.nan # Створення пустої колонки "Назва" для подальшого обєднання з датафреймом з номерами
df_tenders_not_by_schools_numerated = df_tenders_not_by_schools_numerated[col_names]

##### Step 8: Найменування шкіл-не-замовників

In [ ]:
"""
Завантажити датафрейм МОН власних назв навчальниз закладів, підготовлений у "Fileter MON data.ipynb"
school_names - список унікальних назв 
"""
df_names_clean = pandas.read_csv('data/df_names_clean.csv', sep = ';')
df_names_clean = df_names_clean[df_names_clean['Назва'].notnull()] # Видалити NaN
school_names = df_names_clean['Назва'].unique().tolist()

In [ ]:
"""
df_step8 - датафрейм копія закупівель шкіл-не-замовників, де не визначився номер
"""
df_step8 = df_step7_numerated_undefined_4.copy()

In [ ]:
"""
Видалення з df_step8 всіх "технічних" колонок
"""
col_names_temp_8 = training_data_columns.copy() 
col_names_temp_8 += ['Тип закладу', 'Номер']
df_step8 = df_step8[col_names_temp_8]

In [ ]:
"""
Визначення у датафреймі df_step8 власних назв функцією naming (деталі див. у data/functions_dataframe.py)
"""
df_named = naming(df_step8, ['Тендер', 'Лот', 'ОписаниеТендера'], reg_double_quotes, 'naming')

In [ ]:
"""
Фільтрування дафафрейму df_named ключовими словами з бази МОН school_names

Паттерн '^({})$' - шукає повне співпадіння

Ключові слова шукаються в колонках у такому порядку:
        Тендер_naming_first
        Лот_naming_first
        ОписаниеТендера_naming_first
        Тендер_naming_last
        Лот_naming_last
        ОписаниеТендера_naming_last
Якщо є співпадення - рядок вважається назвою закладу. Інші отримують статус "Невідомо".
Для кожної колонки створюється свій датафрейм тільки з тими рядками, де були знайдені ключові слова.
"""

# Тендер_naming_first
df_named['Назва'] = np.where(df_named['Тендер_naming_first'].astype(str).str.contains(r'^({})$'.format('|'.join(school_names)), case = False, na = False, regex = True),
        df_named['Тендер_naming_first'], "Невідомо")
df_named_tender = df_named[df_named['Назва'] != 'Невідомо']
df_named_undefined_1 = df_named[df_named['Назва'] == 'Невідомо']

# Лот_naming_first
df_named_undefined_1['Назва'] = np.where(df_named_undefined_1['Лот_naming_first'].astype(str).str.contains(r'^({})$'.format('|'.join(school_names)), case = False, na = False, regex = True),
        df_named_undefined_1['Лот_naming_first'], "Невідомо")
df_named_lot = df_named_undefined_1[df_named_undefined_1['Назва'] != 'Невідомо']
df_named_undefined_2 = df_named_undefined_1[df_named_undefined_1['Назва'] == 'Невідомо']

# ОписаниеТендера_naming_first
df_named_undefined_2['Назва'] = np.where(df_named_undefined_2['ОписаниеТендера_naming_first'].astype(str).str.contains(r'^({})$'.format('|'.join(school_names)), case = False, na = False, regex = True),
        df_named_undefined_2['ОписаниеТендера_naming_first'], "Невідомо")
df_named_opystendera = df_named_undefined_2[df_named_undefined_2['Назва'] != 'Невідомо']
df_named_undefined_3 = df_named_undefined_2[df_named_undefined_2['Назва'] == 'Невідомо']

# Тендер_naming_last
df_named_undefined_3['Назва'] = np.where(df_named_undefined_3['Тендер_naming_last'].astype(str).str.contains(r'^({})$'.format('|'.join(school_names)), case = False, na = False, regex = True),
        df_named_undefined_3['Тендер_naming_last'], "Невідомо")
df_named_tender_last = df_named_undefined_3[df_named_undefined_3['Назва'] != 'Невідомо']
df_named_undefined_4 = df_named_undefined_3[df_named_undefined_3['Назва'] == 'Невідомо']

# Лот_naming_last
df_named_undefined_4['Назва'] = np.where(df_named_undefined_4['Лот_naming_last'].astype(str).str.contains(r'^({})$'.format('|'.join(school_names)), case = False, na = False, regex = True),
        df_named_undefined_4['Лот_naming_last'], "Невідомо")
df_named_lot_last = df_named_undefined_4[df_named_undefined_4['Назва'] != 'Невідомо']
df_named_undefined_5 = df_named_undefined_4[df_named_undefined_4['Назва'] == 'Невідомо']

# ОписаниеТендера_naming_last
df_named_undefined_5['Назва'] = np.where(df_named_undefined_5['ОписаниеТендера_naming_last'].astype(str).str.contains(r'^({})$'.format('|'.join(school_names)), case = False, na = False, regex = True),
        df_named_undefined_5['ОписаниеТендера_naming_last'], "Невідомо")
df_named_opystendera_last = df_named_undefined_5[df_named_undefined_5['Назва'] != 'Невідомо']

In [ ]:
"""
df_tender_not_by_schools_unidentified - датафрейм шкіл-не-замовників, де не визначився ані номер, ані назва
"""
df_tender_not_by_schools_unidentified = df_named_undefined_5[df_named_undefined_5['Назва'] == 'Невідомо']
df_tender_not_by_schools_unidentified = df_tender_not_by_schools_unidentified[col_names]

In [ ]:
"""
df_tenders_not_by_schools_named - об'єднаний датафрейм шкіл-не-замовників з номерами
Видалення "технічних" колонок, створених функціями.
"""
frames = [ 
            df_named_tender, 
            df_named_lot,
            df_named_opystendera, 
            df_named_tender_last, 
            df_named_lot_last, 
            df_named_opystendera_last
         ]
df_tenders_not_by_schools_named = pd.concat(frames)
df_tenders_not_by_schools_named = df_tenders_not_by_schools_named[col_names]

##### Step 9: Об'єднання двох датафреймів шкіл-не-замовників

In [ ]:
"""
df_tenders_not_by_schools - датафрейм шкіл-не-замовників з визначеними номерами і назвами
Створення колонки 'Заклад-замовник'
"""
frames = [
    df_tenders_not_by_schools_numerated, 
    df_tenders_not_by_schools_named
]
df_tenders_not_by_schools = pd.concat(frames)
df_tenders_not_by_schools['Заклад-замовник'] = False

##### Step 9.5: Створення єдиної колонки для Номеру і Назви - Номер/Назва (ЩЕ ПІД ПИТАННЯМ)

In [ ]:
# """
# У датафреймі Шкіл-замовників df_tenders_by_schools_temp в рядках, 
#     - де Номер не визначився, присвоюється Назва
#     - де Назва не визначилась, присвоюється поле Організатор
# """

# df_tenders_by_schools_temp = df_tenders_by_schools.copy()
# df_tenders_by_schools_temp['Номер/Назва'] = np.where(df_tenders_by_schools_temp['Номер'].notnull(), df_tenders_by_schools_temp['Номер'], df_tenders_by_schools_temp['Назва'])
# df_tenders_by_schools_temp['Номер/Назва'] = np.where(df_tenders_by_schools_temp['Номер/Назва'].isnull(), df_tenders_by_schools_temp['Организатор'], df_tenders_by_schools_temp['Номер/Назва'])

In [ ]:
# """
# У датафреймі Шкіл-не-замовників df_tenders_not_by_schools в рядках, 
#     - де Номер не визначився, присвоюється Назва
#     - де Назва не визначилась, присвоюється поле Організатор

# В колонці 'Номер/Назва' значення "Невідомо" фактично означає закупівлю для кількох закладів освіти

# """

# df_tenders_not_by_schools_temp = df_tenders_not_by_schools.copy()
# df_tenders_not_by_schools_temp['Номер/Назва'] = np.where(df_tenders_not_by_schools_temp['Номер'].notnull(), df_tenders_not_by_schools_temp['Номер'], df_tenders_not_by_schools_temp['Назва'])

##### Step 10: Об'єднання датафреймів шкіл-не-замовників та шкіл-замовників

In [ ]:
frames = [df_tenders_by_schools, df_tenders_not_by_schools]
df_tenders = pd.concat(frames)
print("df_tenders", df_tenders.shape)

In [ ]:
df_tenders_filtered_1 = df_tenders.sample(1000)
df_tenders_filtered_1['Correct'] = 'undefined'
print("df_tenders_filtered_1:", df_tenders_filtered_1.shape, "\nCols:", df_tenders_filtered_1.columns)
# df_tenders_filtered_1.to_csv('results/school-procurements-1000-random.csv', sep=';')

#### Step 11: RandomDataResuls

In [ ]:
random_1000_results = pandas.read_csv('results/school-procurements-1000-random-marked.csv', index_col = 0, error_bad_lines=False, sep = ',', low_memory = False)
random_1000_results = random_1000_results.iloc[:, :-1]
print(random_1000_results.shape)

In [ ]:
""" UNDONE

    Correct mark #21 (відсутній символ №):
        Зміни у Step 7:
            мабуть рішенням є регулярний вираз, який би відбирав 1-3 значне число,
            яке знаходить відразу після ключового слова.
            Можливо відстань рахувати між знайденими ключовими словами?
                приклад index 4337: "Капітальний ремонт будівлі ДНЗ 126"
                
    
    replace 'ДНЗ ' - 'ДНЗ №'
            
"""
step11 = random_1000_results[random_1000_results.Correct=='21']

In [ ]:
""" UNDONE

    Correct mark #22 (кілька символів №):
        Зміни у Step 7:
            мабуть рішенням є регулярний вираз, який би шукав число з символом №,
            яке знаходиться ПІСЛЯ знайденого ключового слова, а не ДО. І він має бути тільки один.
            Можливо відстань рахувати між знайденими ключовими словами?
                приклад index 78075: "Лот №58 крупи та борошно, НВК №8"
            
"""
random_1000_results[random_1000_results.Correct=='22']

In [ ]:
"""
TODO
"""

In [ ]:
""" UNDONE

    Correct marks #26 (кілька лапок): 
        Для шкіл-не-замовників у Step 8
        1. «» або ""
        2. відібрати перше? останнє? повторюване?

"""
random_1000_results[(random_1000_results.Correct=='25') | (random_1000_results.Correct=='26')]